# An Angry Birds adaptation in Python using VPython: 18018699

In the following Jupiter notebook we have used the Numpy and VPython libraries in order to create a game resembling the same mechanics as that of "Angry Birds". In the following notebook we begin by setting up our "scene" environment and "ground", which are going to be constant in all instances of the game. 

In the case of the game we will have our bird as being a Python ball object and our target as being a VPython cuboid object. We will then use the function "random()" in order to create a new and different environment for each time the game is loaded into a new instance. 

The main aim of the game is going to be to first input an initial speed and launch angle with the aim of hitting the target and causing it to topple over. If these initial parameters inputed by the user lead to the target not toppling over, due to the force from the bird not being large enough or the bird simply missing the target, then the game will restart in a new instance so that the user can have another chance of toppling the target

To achieve this aim we will begin by defining some of the parameters of both our target and our bird, this includes things like the mass and parameters relating to the animation of the objects in our VPython scene. We will then define a function which will run a new instance of the game eveytime it is called.

Each instance begins by setting new parameters using the "random()" function, which create new and original enviorments. Once those parameters are set the game will then ask for the user to input their initial velocity and angle of launch. Using the values inuted by the user, the program will then use a while fucntion to run the animation of the bird and calculate the path of the bird. Using the calcultions of the path the function will then use an if statement to test it the ball has hit the target. If the ball has hit the target we then refer to a nested if statement to test if this collision would cause the target to topple. If it does cause the target to topple, tha target will then be subject to rotate 90 degrees clockwise about the the bottom right corner. If it doesn't cause the target to topple or does not even hit the target this will lead to the program creating a new instance for the player to try again until they win a game.

In [1]:
#Importing the libraries we will be needing in this notebook.
import numpy as np
#Importing the necessary VPython functions which we will be using in the program
from vpython import sphere, color, rate, canvas, vector, curve, label, box, cross, mag, random, arrow

###PARAMETERS###

#Defining the small change in time which will be used in the animations of the bird
dt = 0.001

#Gravitaitonal acceleration of the Earth
g = -9.81#metres/sec^2, 2.d.p.

#Dimensions of the target
target_length = 0.5 #metres
target_height = 2 #metres
target_width = 0.5 #metres
target_mass = 100 #kg

#Parameters of the platform
platform_length_half = 0.5 #metres

#The parameters of the bird
bird_mass = 0.1 #kg
bird_radius_calculations = 0.05 #metres. This radius will be used in all the back-end calculations.
bird_radius_animation = 0.3 #metres. This radius will be used when showing the bird in animations.

#TIme for the collision between the bird and the target
delta_t_collision = 0.01 #sec

#The function which initialises an instance of the game
def game():
    
    #Creating the background of the whole game
    scene = canvas(width=640, 
                   height=480,
                   center=vector(8,5,0),
                   range=8)

    #Creating the ground
    ground = curve(pos=[(0,0,0),(16,0,0)],
                       color=color.green)
    
    #This statement will be used as a way to leave the game and end the program once the user has won.
    want_to_play = True
    
    while want_to_play == True: 
        #Creaing Platform for the bird
        #First we set the variable "platform_height" to be equal to a random number between 1 and 0
        platform_height = random()

        #Here We have finally defined our platform using the variables we have set above.
        platform = curve(pos=[((-platform_length_half),platform_height,0),(platform_length_half,platform_height,0)], 
                         color=color.white)

        #Target parameters
        x_random_centre = 10*random() + 5 #metres 
        target_x_centre = x_random_centre #metres
        target_y_centre = target_height/2 #metres
        target_z_centre = 0 #metres
        target_position_vector = vector(target_x_centre, target_y_centre, target_z_centre)

        #Create target
        target = box(pos=target_position_vector,
                     length=target_length,
                     height=target_height,
                     width=target_width)

        #Initial Position of the bird
        bird_x0 = 0 #metres
        bird_y0 = platform_height + bird_radius_calculations #metres
        bird_z0 = 0 #metres

        #Create bird
        bird = sphere(pos=vector(bird_x0, bird_y0, bird_z0),
                      radius=bird_radius_animation, 
                      color=color.red, 
                      make_trail = True)
        
        #Initialising the parameters used to to define the position of the bird
        bird_x = bird_x0
        bird_y = bird_y0
        bird_z = bird_z0
        
        #Create Momentum vector
        momentum_vec = arrow(pos = vector(bird_x,bird_y,bird_z),
                      shaftwidth= 0.1,
                      color=color.blue)

        #We will now ask the user for the values for the initial velocity of the bird
        v0 = float(input("What velocity (m/s) would you like to launch your bird at? "))
        angle0_degrees = float(input("What angle (degrees) would you like to launch your bird at? "))
        angle0_radians = angle0_degrees * (np.pi / 180)
        angle0 = angle0_radians
        
        #We will seperate this initial velocity into its horizontal(x) and vertical(y) components 
        v_x0 = v0 * np.cos(angle0) #m/s
        v_y0 = v0 * np.sin(angle0) #m/s
        
        #Before we begin the animation we must set the time "t" equal to 0
        t = 0 #sec
        
        #For the animation of the ball we will use this while function to calculate the path of the bird
        while bird_x <= (target_x_centre - target_length/2 - bird_radius_calculations) and bird_y > 0:

            #The rate at which the animation runs at
            rate(250)

            #Here we are updating the time for every itteration through the while loop by a small
            #amount od time dt
            t = t + dt #sec

            #Using the updated time to calculate the velocity of the bird at that moment in time
            v_x = v_x0 #m/s, horizontal component 
            v_y = v_y0 + g*t #m/s, vertical component 

            #Calculating the new x and y position values of the bird
            bird_x = bird_x0 + v_x0 * t #metres, horizontal component 
            bird_y = bird_y0 + v_y0 * t  + ((0.5*g*(t**2))) #metres, vertical component

            #Updating the position vector of the bird
            bird.pos = vector(bird_x,bird_y,bird_z)
            
            #Updating the momentum values in the x and y directions
            p_x = bird_mass * v_x #kg m/s, horizontal component 
            p_y = bird_mass * v_y #kg m/s, vertical component

            #Updating the momentum vector arrow with the bird's new position and new momentum values
            momentum_vec.pos = bird.pos
            momentum_vec.axis = vector(p_x,p_y,0)

        #If statement to check if the bird's path causes it to hit the target
        if bird_y < target_height + bird_radius_calculations and bird_y > 0:
            
            ###YES it does hit the target###
            
            #Horizontal distance between the point of rotation and the centre of mass.
            d_r = vector((0.5*target_width),0,0)
            #The vector representing the weight of the target
            F_grav = vector(0,(target_mass*g),0)
            #Calculating the restoring torque of the target
            T_restoring = cross(F_grav,d_r)
            
            #Vector from the point of rotation to the point of impact
            d_a = bird.pos - (target.pos - vector(-target_length/2, target_height/2, 0))
            #Force applied by the change in momentum of the target, through the bird transfering all it's momentum to the target
            F_applied = vector(p_x,p_y,0) / delta_t_collision
            #Calculating the applied torque on the target
            T_applied = cross(F_applied,d_a)
            
            #If statement to check if the bird after hitting the target would topple the target.
            if mag(T_applied) > mag(T_restoring):
                
                ###It WOULD topple the target###
                
                #Since the applied torque's magnitude exceeds that of the restoring torque this means that target will topple
                #to animate the toppling and show the user this we will use the "rotate" function to rotate the target to a position where
                #it has been toppled.
                target.rotate(angle=-np.pi/2,
                              axis=vector(0,0,1),
                              origin=(target.pos + vector(target_length/2,-target_y_centre,0)))
                
                print("Congratulations you won the game!!! :)")
                
                #Printing facts about the collision
                print("The height of the impact point was " + str(bird_y) + " metres.")
                print("The bird’s momentum at the point of impact was " + str(mag(vector(p_x,p_y,0))) + " kg m/s.")
                print("The applied torque " + str(T_applied) + " Nm.")
                print("The magnitude of the restoring torque " + str(mag(T_restoring)) + " Nm.")
                
                #Ending the program by setting want_to_play to False, ending the while loop.
                want_to_play = False
            
            else:                
                 
                ###It WOULD'NT topple the target###

                print("Sorry you did not hit the target at a high enough speed. You will now be trying again.")
                #Starting new instance of the game for the user to retry
                game()
        
        else:
            
            ###NO it does not hit the target###
            
            print("Sorry you did not hit the target. You will now be trying again.")
            #Starting new instance of the game for the user to retry
            game()
        
        #Ending the function
        return;

#Running the function   
game()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

What velocity (m/s) would you like to launch your bird at? 20
What angle (degrees) would you like to launch your bird at? 40
Sorry you did not hit the target. You will now be trying again.


<IPython.core.display.Javascript object>

What velocity (m/s) would you like to launch your bird at? 19
What angle (degrees) would you like to launch your bird at? 10
Congratulations you won the game!!! :)
The height of the impact point was 1.40822017945 metres.
The bird’s momentum at the point of impact was 1.8726715643630474 kg m/s.
The applied torque <0, -0, 267.593> Nm.
The magnitude of the restoring torque 245.25 Nm.


# Conclusion

## How you would suggest improving the representation of the physics in the game?

In the case of this game we have assumed that if the bird hits the target that all of the momentum is transfered to the target as well as the the collision being completely elastic. This is not an accurate representation of a real world scenario since most collisions are not totally elastic but rather inelastic. We have also excluded the fact that in a real world scenario most bodies travelling through air will have their path significantly effected by the air resistance.

We have also not accurately animated the toppling of the target itself. We have instead simply forced the target to rotate once the applied torque exceeds the restoring torque. This does not accurately show how this would happen in the real world, with the speed at which the target topples being dependent on how much the restoring torque was exceeded. We have also not animated the case in which the restoring torque exceeds the applied torque as the target begins to rotate slightly and then goes back to it's original position.

## How you might go about implementing these improvements in Python?

Implementing the air resistance would come from using the fact that their is a force $ F_{air} = -\beta v $. This force is directly in opposition to the motion of the bird. By using an accepted value of $ \beta $ and then taking horizonatal and vertical components of the force we can then accodunt for this force. Dividing the force by the mass of the bird to get acceleration terms which will then be used in the calculations of the velocity, and also position of the ball $ a_{air} = \frac{-\beta v}{m_{bird}} $.

The toppling can be accounted for by using the rotate function in itterations inside of a while loop after having calculeted the time taken for the target to topple .